In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import replicate
import json

In [4]:
from pydub import AudioSegment
import io

This is the first 25ish minutes of the webinar

In [5]:
# audio = AudioSegment.from_file("./data/youtube/langchain-openai-webinar-clip.mkv")

In [6]:
def getFileNames(splitNumber):
    fileId = "langchain-webinar first split {}".format(splitNumber)
    return "./data/youtube/{}.mp3".format(fileId), "./data/youtube/{} - Replicate x Whisper API Response.json".format(fileId)

In [87]:
chunk_duration = 10
chunk_duration_ms = chunk_duration * 60 * 1000

In [7]:
# for split_number, i in enumerate(range(0, len(audio), chunk_duration_ms)):
# 	# Audio chunk
# 	chunk = audio[i : i + chunk_duration_ms]

# 	audioFilePath, responseFilePath = getFileNames(split_number)

# 	chunk.export(audioFilePath, format="mp3")

# 	output = replicate.run(
# 		"openai/whisper:91ee9c0c3df30478510ff8c8a3a545add1ad0259ad3a9f78fba57fbc05ee64f7",
# 		input={"audio": open(audioFilePath, "rb")}
# 	)
# 	with open(responseFilePath, 'w') as f:
# 		json.dump(output, f)

In [94]:
segments = []
for i in range(3):
	_, responseFilePath = getFileNames(i)
	output = json.load(open(responseFilePath, 'r'))
	for segment in output['segments']:
		moddedSegment = segment
		moddedSegment['start'] += i * chunk_duration * 60
		moddedSegment['end'] += i * chunk_duration * 60
		segments.append(moddedSegment)

In [96]:
transcript = "\n".join([segment['text'] for segment in segments])
transcript = transcript.strip()

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [97]:
textSplitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [98]:
transcriptChunks = textSplitter.create_documents([transcript])

Add consecutive segments of the video as `metadata` to each chunk of the transcript

In [99]:
SEGMENT_OVERLAP_SECONDS = 1
chunksWithSegments = []

for chunk in transcriptChunks:
    chunkText = chunk.page_content

    chunkSegments = []
    for segment in segments:
        if segment['text'].strip().lower() in chunkText.strip().lower():
            chunkSegments.append(segment)

    # Store consecutive segments as one segment
    consecutiveSegments = [{
        'start': chunkSegments[0]['start'],
        'end': chunkSegments[0]['end']
    }]

    for segment in chunkSegments[1:]:
        lastSegmentEnd = consecutiveSegments[-1]['end']

        # range doesn't work for floats
        if segment['start'] >= lastSegmentEnd and segment['start'] <= lastSegmentEnd + SEGMENT_OVERLAP_SECONDS:
            consecutiveSegments[-1]['end'] = segment['end']
        else:
            consecutiveSegments.append({
                'start': segment['start'],
                'end': segment['end']
            })
    
    chunksWithSegments.append(Document(
        page_content=chunkText,
        metadata={
        'blockId': "k3u46gu4bg",
        'segments': json.dumps(consecutiveSegments)
        })
    )


In [100]:
for c in chunksWithSegments:
    if 'sql' in c.page_content.lower():
        print(c)

page_content="think these are some cool examples worth showing off. The first example I'll show\n is one around maybe preventing SQL injection. Right now when we use SQL\n agents, they kind of just output the SQL with like no escaped values. But we know\n that to call SQL safely, we want to have template strings and we want to have\n query parameters. So if we model this, so you know, a SQL template has a literal\n or an identifier. There are parameters that have keys and values with a type." metadata={'blockId': 'k3u46gu4bg', 'segments': '[{"start": 1225.12, "end": 1255.04}]'}
page_content="And the SQL query is actually a template and its parameters. And just for\n safety, I add a is dangerous flag that determines whether or not I think or the\n model thinks there's any kind of SQL injection. So this is all kind of just\n modeling the data. What PyData can do is it generates the function schema\n automatically for you. Here I just give an example of some SQL tables and then I\n make a

In [101]:
# db = Chroma.from_documents(documents=chunksWithSegments, embedding=OpenAIEmbeddings(), persist_directory="./langchain_webinar_db")
# db.persist()
# db = None

In [102]:
db = Chroma(persist_directory="./langchain_webinar_db", embedding_function=OpenAIEmbeddings())

In [105]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo-0613"),
    chain_type="stuff",
    retriever=db.as_retriever(),
    return_source_documents=True
)

In [106]:
res = qa("What was the point about sql injection?")
print(res['result'])

The point about SQL injection was that the speaker mentioned the importance of using template strings and query parameters to safely call SQL queries, in order to prevent SQL injection. They also mentioned that the data model they showed included a flag to determine if there was any potential for SQL injection.


In [110]:
res['source_documents'][0].metadata

{'blockId': 'k3u46gu4bg', 'segments': '[{"start": 1225.12, "end": 1255.04}]'}